# 4.05 Data consistency Checks
## Importing libraries and csv files

In [24]:
import numpy as np
import pandas as pd
import os

pathData = r'C:\Users\Michael\Desktop\Career Foundry\02 Data Immersion Course\04 Python Fundamentals for Data Analysts\Instacart Basket Analysis 2023 11\02 Data'
df_prods = pd.read_csv(os.path.join(pathData, 'Original Data', 'products.csv'), index_col = False)
df_ords = pd.read_csv(os.path.join(pathData, 'Prepared Data', 'orders_wrangled.csv'), index_col = False)

# 01 Cleaning data from products.csv

### Finding any Null values and removing the rows with Null for product_name

In [44]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

In [51]:
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [53]:
df_prods_clean.shape

(49677, 5)

## Finding and removing duplicate rows

In [54]:
df_prods_clean[df_prods_clean.duplicated()]

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [55]:
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [57]:
df_prods_clean_no_dups.shape

(49672, 5)

## Saving the new products csv file to then Prepared Data folder

In [60]:
df_prods_clean_no_dups.to_csv(os.path.join(pathData, 'Prepared Data', 'products_checked.csv'), index = False)

# 02 Describe the orders dataframe
## Changing the id values to string first as these should be qualitative data

In [76]:
df_ords['order_id'] = df_ords['order_id'].astype('str')
df_ords['user_id'] = df_ords['user_id'].astype('str')

# change options to display float type, not scienctific notation
pd.options.display.float_format = '{:.2f}'.format

# describe the Orders dataframe
df_ords.describe()

,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
count,3421083.00,3421083.00,3421083.00,3214874.00
mean,17.15,2.78,13.45,11.11
std,17.73,2.05,4.23,9.21
min,1.00,0.00,0.00,0.00
25%,5.00,1.00,10.00,4.00
50%,11.00,3.00,13.00,7.00
75%,23.00,5.00,16.00,15.00
max,100.00,6.00,23.00,30.00


### Findings from data description
All have the same count.

Order_number: Ranges from 1 to 100, with a median of 11. LQ = 5, UQ = 23, IQR = 18. these values all make sence in the contect of the number of orders that an individual customer would have made.

Order_day_of_week: Ranges from 0 to 6, representing the seven days as outlined in the Project Brief (0= Sat, 1 = Sun, ..., 6 = Fri)

Order_hour_of_day: Ranges from 0 to 23 representing the 24 hours of the day. A mean of 13.45 represents an average order time of ~13:30 and the median order time is 13:00.

Days_since_last_order: ranges from 0 to 30, with a median of 7. Meaning some people have made two orders in a day, with a maximum of 30 days between orders (one month) and averaging a weekly order. The mean is higher that the median, which is consistent with the high maximum relative to the median.

None of these data appear incorrect from the described data.

# 03 Mixed Data Types in Orders

In [95]:
# Parsing all columns or orders in FOR LOOP
for col in df_ords.columns.tolist():
# Variable search_mixed is
  search_mixed = (df_ords[[col]].map(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
## Output if mixed, or in not mixed dive the type
  if len (df_ords[search_mixed]) > 0:
    print (col + 'is mixed')
  else:
    print (col + ' is not mixed. | ' + 'Type is ' + str(df_ords[col].dtype))

order_id is not mixed. | Type is object
user_id is not mixed. | Type is object
order_number is not mixed. | Type is int64
orders_day_of_week is not mixed. | Type is int64
order_hour_of_day is not mixed. | Type is int64
days_since_last_order is not mixed. | Type is float64


### Outcomes from data parsing
There are no mixed data types in the orders dataframe.

# 05 Missing data

In [98]:
df_ords.isnull().sum()

order_id                      0
user_id                       0
order_number                  0
orders_day_of_week            0
order_hour_of_day             0
days_since_last_order    206209
dtype: int64

In [102]:
# Reviewing the null entries to determine the reason for their presence
df_ords[df_ords['days_since_last_order'].isnull() == True]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order
0,2539329,1,1,2,8,NaN
11,2168274,2,1,2,11,NaN
26,1374495,3,1,1,14,NaN
39,3343014,4,1,6,11,NaN
45,2717275,5,1,3,12,NaN
...,...,...,...,...,...,...
3420930,969311,206205,1,4,12,NaN
3420934,3189322,206206,1,3,18,NaN
3421002,2166133,206207,1,6,19,NaN
3421019,2227043,206208,1,1,15,NaN


# 06 Addressing missing values

Leave these values, they represent first orders that a customer has made (see the order number . Therefore being blank is correct (0 would represent a 2nd order in the same day).

# 07 Finding duplicates

In [100]:
df_ords[df_ords.duplicated()]

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order


No duplicates found

# 09 Export the data to the Prepared data file

In [104]:
df_prods_clean_no_dups.to_csv(os.path.join(pathData, 'Prepared Data', 'products_checked.csv'), index = False)
df_ords.to_csv(os.path.join(pathData, 'Prepared Data', 'orders_checked.csv'), index = False)